#Import Libraries

In [0]:
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

#Check GPU, CPU Specifications

In [0]:
tf.test.gpu_device_name() #Check whether GPU is working or not
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4815002629348805237, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 3065138538133984223
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9701166393815194390
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11150726272
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 2677822464525206342
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [0]:
!cat /proc/meminfo  #Check the RAM used

MemTotal:       13333556 kB
MemFree:         3883652 kB
MemAvailable:   11161136 kB
Buffers:          130280 kB
Cached:          6843600 kB
SwapCached:            0 kB
Active:          3119008 kB
Inactive:        5850140 kB
Active(anon):    1603956 kB
Inactive(anon):     5452 kB
Active(file):    1515052 kB
Inactive(file):  5844688 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               856 kB
Writeback:             0 kB
AnonPages:       1995164 kB
Mapped:           768988 kB
Shmem:              6076 kB
Slab:             291344 kB
SReclaimable:     243104 kB
SUnreclaim:        48240 kB
KernelStack:        4604 kB
PageTables:        12384 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6666776 kB
Committed_AS:    4697592 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:              936 kB
AnonHugePages:   

In [0]:
!cat /proc/cpuinfo  #Check the CPU info

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs itlb_multihit
bogomips	: 4600.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management

#Build and Train the model

In [0]:
#Callback for stopping the training process if training accuracy goes beyond the DESIRED_ACCURACY
DESIRED_ACCURACY = 0.95

class myCallback(tf.keras.callbacks.Callback):# your code):
        def on_epoch_end(self, epoch, logs = {}):
            if(logs.get('accuracy')>DESIRED_ACCURACY):
                print("Reached greater than 95.0% accuracy so cancelling training!")
                self.model.stop_training = True

callbacks = myCallback()  #Initiating the callback object

# This Code Block defines and compiles the Model. We are assuming the images are 600 X 600 in the implementation.
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, kernel_size=(3, 3), kernel_initializer="he_normal", padding="same", input_shape=(600, 600, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Conv2D(16, kernel_size=(3, 3), kernel_initializer="he_normal", padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), kernel_initializer="he_normal", padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), kernel_initializer="he_normal", padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.5),
    
    
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), kernel_initializer="he_normal", padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), kernel_initializer="he_normal", padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.5),
    

    tf.keras.layers.Conv2D(128, kernel_size=(3, 3), kernel_initializer="he_normal", padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Conv2D(128, kernel_size=(3, 3), kernel_initializer="he_normal", padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.5),
    

    tf.keras.layers.Conv2D(128, kernel_size=(3, 3), kernel_initializer="he_normal", padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Conv2D(128, kernel_size=(3, 3), kernel_initializer="he_normal", padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.5),
    

    tf.keras.layers.Conv2D(256, kernel_size=(3, 3), kernel_initializer="he_normal", padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Conv2D(256, kernel_size=(3, 3), kernel_initializer="he_normal", padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.5),
    
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 2048 neuron hidden layer
    tf.keras.layers.Dense(2048, activation='relu'),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('pneumonia') and 1 for the other ('normal')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()


model.compile(loss='binary_crossentropy', optimizer= 'adam', metrics=['accuracy'])
    

# This code block creates an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

train_datagen = ImageDataGenerator(rescale = 1/255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
# A target_size of 600 X 600 is used.
train_generator = train_datagen.flow_from_directory(
    '/content/drive/My Drive/Colab Notebooks/SmartInternz/ChestXray/chest_xray/train',    #Paste the PATH of the train folder here
    target_size=(600, 600),  # All images will be resized to 600x600
    batch_size=4,
    class_mode = 'binary')


# This code block calls model.fit_generator and train for a number of epochs for model fitting.
history = model.fit_generator(train_generator,
  steps_per_epoch=512,  
  epochs=15,
  verbose=1, callbacks = [callbacks])


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 600, 600, 16)      448       
_________________________________________________________________
batch_normalization_36 (Batc (None, 600, 600, 16)      64        
_________________________________________________________________
activation_36 (Activation)   (None, 600, 600, 16)      0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 600, 600, 16)      2320      
_________________________________________________________________
batch_normalization_37 (Batc (None, 600, 600, 16)      64        
_________________________________________________________________
activation_37 (Activation)   (None, 600, 600, 16)      0         
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 300, 300, 16)     

In [0]:
#Saving the model into a .h5 file
model.save('/content/drive/My Drive/Colab Notebooks/SmartInternz/ChestXray/my_model.h5')